In [36]:
from PriceLoader import test

print(test())

123


List of S&P 500 companies

In [37]:
import pandas as pd
import requests
from io import StringIO

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

response = requests.get(url, headers=headers)

tables = pd.read_html(StringIO(response.text))
sp500_df = tables[0]


In [38]:
sp500_df['Symbol'].to_csv('sp500_symbols.csv', index=False)

sp500_df['Symbol']

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

Import list and download data

In [ ]:
import yfinance as yf

def batch_download_data(sp500_symbols, batch_size = 15, start_date = '2020-01-01', end_date = '2025-01-01'):

    all_ticker_data = []

    for i in range(0, len(sp500_symbols), batch_size):

        batch = sp500_symbols[i:i+batch_size]
        tickers = batch.iloc[:, 0].tolist()

        df = yf.download(tickers, start=start_date, end=end_date)

        if len(tickers) == 1:
            batch_data = {
                    'Close': df['Close'].to_frame(tickers[0]),
                    'Volume': df['Volume'].to_frame(tickers[0])
                }
        else:   
                # For multiple tickers, extract Close and Volume separately
            batch_data = {
                'Close': df['Close'],
                'Volume': df['Volume']
            }

        all_ticker_data.append(batch_data)
    
    combined_close = pd.concat([data['Close'] for data in all_ticker_data], axis=1)
    combined_volume = pd.concat([data['Volume'] for data in all_ticker_data], axis=1)
    
    return {'Close': combined_close, 'Volume': combined_volume}

In [40]:
def save_combined_dataframe_to_parquet(combined_df, data_dir='data'):
    
    for ticker in combined_df['Close'].columns:
        try:
            ticker_data = pd.DataFrame({
                'Date': combined_df['Close'].index,
                'Adj_Close': combined_df['Close'][ticker].values,
                'Volume': combined_df['Volume'][ticker].values
            }).reset_index(drop=True)
            
            filename = f"{data_dir}/{ticker}.parquet"
            ticker_data.to_parquet(filename, index=False)
            print(f"Saved {ticker}: {len(ticker_data)} records to {filename}")
            
        except Exception as e:
            print("Error")

In [41]:
import pandas as pd
from io import StringIO

df_sp500 = pd.read_csv('sp500_symbols.csv')
# df_sp500

test_set = df_sp500.head(20)
# test_set

numeric_data = batch_download_data(test_set, start_date = '2010-01-01', end_date = '2025-01-01')

numeric_data['Volume']
# numeric_data['Close']

/var/folders/ty/5gy6m3k52ps97ms9_x3glvnh0000gn/T/ipykernel_89099/3115139777.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  15 of 15 completed
/var/folders/ty/5gy6m3k52ps97ms9_x3glvnh0000gn/T/ipykernel_89099/3115139777.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=start_date, end=end_date)
[*********************100%***********************]  5 of 5 completed


Ticker,A,ABBV,ABNB,ABT,ACN,ADBE,AES,AFL,AKAM,ALB,AMD,AOS,APD,ARE,MMM,ALGN,ALL,ALLE,GOOGL,LNT
Date,,,,,,,,,,,,,,,,,,,,
2010-01-04,3815561,NaN,NaN,10829095,3650100,4710200,4597600,4724000,3015600,712900,18748700,1104600,1131483,467100,3640265,374200,2631000,NaN,78169752,1332800
2010-01-05,4186031,NaN,NaN,10562109,2613000,7108800,5093200,7930600,5421900,398900,22145700,1207200,790103,527000,3405012,431900,5342100,NaN,120067812,3684600
2010-01-06,3243779,NaN,NaN,11401417,5772100,5336400,7507300,5851600,2863900,717200,18643400,663000,1248663,566700,6301126,667000,3399500,NaN,158988852,2092400
2010-01-07,3095172,NaN,NaN,12857232,4022900,5576700,16843500,5535800,2919400,603700,26806800,564000,1514481,850900,5346240,450000,3694900,NaN,256315428,1404400
2010-01-08,3733918,NaN,NaN,12148604,5069700,5429200,5647500,4393400,2106600,759000,13752800,504600,1244880,665200,4073337,675200,2806800,NaN,188783028,2466200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,370200,2692800.0,1677100.0,1480000,1075400,1685000,3967400,901300,541700,595100,24829600,381800,296100,425800,803200,445600,453200,248800.0,10403300,397300
2024-12-26,556600,2759300.0,2067000.0,2960500,910500,2131200,7272100,2491000,946900,1194900,24998200,670600,418500,641700,1484900,570900,563300,386700.0,12046600,844500
2024-12-27,631800,2745400.0,2650100.0,2829500,1195700,2947200,9280700,1104800,724900,1397000,32924000,517900,457000,1305500,1842600,390400,1094400,429000.0,18891400,728900


In [42]:
save_combined_dataframe_to_parquet(numeric_data)

Saved A: 3774 records to data/A.parquet
Saved ABBV: 3774 records to data/ABBV.parquet
Saved ABNB: 3774 records to data/ABNB.parquet
Saved ABT: 3774 records to data/ABT.parquet
Saved ACN: 3774 records to data/ACN.parquet
Saved ADBE: 3774 records to data/ADBE.parquet
Saved AES: 3774 records to data/AES.parquet
Saved AFL: 3774 records to data/AFL.parquet
Saved AKAM: 3774 records to data/AKAM.parquet
Saved ALB: 3774 records to data/ALB.parquet
Saved AMD: 3774 records to data/AMD.parquet
Saved AOS: 3774 records to data/AOS.parquet
Saved APD: 3774 records to data/APD.parquet
Saved ARE: 3774 records to data/ARE.parquet
Saved MMM: 3774 records to data/MMM.parquet
Saved ALGN: 3774 records to data/ALGN.parquet
Saved ALL: 3774 records to data/ALL.parquet
Saved ALLE: 3774 records to data/ALLE.parquet
Saved GOOGL: 3774 records to data/GOOGL.parquet
Saved LNT: 3774 records to data/LNT.parquet


In [43]:
import yfinance as yf
import pandas as pd
import os

def test():
    return 123

class PriceLoader:
    def __init__(self):
        pass

    def load_price(self, ticker: str = 'AMD', start_date: str = '2020-01-01', end_date: str = '2025-01-01', data_dir: str = 'data'):

        all_close_data = {}
        all_volume_data = {}

        if ticker == '*':
            for file_name in os.listdir(data_dir):
                if file_name.endswith('.parquet'):
                    ticker = file_name.replace('.parquet', '')
                    file_path = os.path.join(data_dir, file_name)

                    df = pd.read_parquet(file_path)
                    df = df.set_index('Date')
                    all_close_data[ticker] = df['Adj_Close']
                    all_volume_data[ticker] = df['Volume']

        else:
            file_path = os.path.join(data_dir, f"{ticker}.parquet")
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"Missing data file: {file_path}")

            df = pd.read_parquet(os.path.join(data_dir, f"{ticker}.parquet"))
            df = df.set_index('Date')
            all_close_data[ticker] = df['Adj_Close']
            all_volume_data[ticker] = df['Volume']

        combined_close_df = pd.DataFrame(all_close_data)
        combined_volume_df = pd.DataFrame(all_volume_data)

        return {'Close': combined_close_df, 'Volume': combined_volume_df}

In [44]:
x = PriceLoader()
amd_df = x.load_price('AMD')
amd_df

{'Close':                    AMD
 Date                  
 2010-01-04    9.700000
 2010-01-05    9.710000
 2010-01-06    9.570000
 2010-01-07    9.470000
 2010-01-08    9.430000
 ...                ...
 2024-12-24  126.290001
 2024-12-26  125.059998
 2024-12-27  125.190002
 2024-12-30  122.440002
 2024-12-31  120.790001
 
 [3774 rows x 1 columns],
 'Volume':                  AMD
 Date                
 2010-01-04  18748700
 2010-01-05  22145700
 2010-01-06  18643400
 2010-01-07  26806800
 2010-01-08  13752800
 ...              ...
 2024-12-24  24829600
 2024-12-26  24998200
 2024-12-27  32924000
 2024-12-30  30501600
 2024-12-31  30203400
 
 [3774 rows x 1 columns]}

In [47]:
total_df = x.load_price('*')
# total_df['Close']
total_df['Volume']

,ABBV,A,AES,AOS,ALB,ABNB,ACN,APD,AKAM,GOOGL,ADBE,MMM,ALLE,LNT,ALL,ALGN,AFL,ABT,AMD,ARE
Date,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,3815561,4597600,1104600,712900,NaN,3650100,1131483,3015600,78169752,4710200,3640265,NaN,1332800,2631000,374200,4724000,10829095,18748700,467100
2010-01-05,NaN,4186031,5093200,1207200,398900,NaN,2613000,790103,5421900,120067812,7108800,3405012,NaN,3684600,5342100,431900,7930600,10562109,22145700,527000
2010-01-06,NaN,3243779,7507300,663000,717200,NaN,5772100,1248663,2863900,158988852,5336400,6301126,NaN,2092400,3399500,667000,5851600,11401417,18643400,566700
2010-01-07,NaN,3095172,16843500,564000,603700,NaN,4022900,1514481,2919400,256315428,5576700,5346240,NaN,1404400,3694900,450000,5535800,12857232,26806800,850900
2010-01-08,NaN,3733918,5647500,504600,759000,NaN,5069700,1244880,2106600,188783028,5429200,4073337,NaN,2466200,2806800,675200,4393400,12148604,13752800,665200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-24,2692800.0,370200,3967400,381800,595100,1677100.0,1075400,296100,541700,10403300,1685000,803200,248800.0,397300,453200,445600,901300,1480000,24829600,425800
2024-12-26,2759300.0,556600,7272100,670600,1194900,2067000.0,910500,418500,946900,12046600,2131200,1484900,386700.0,844500,563300,570900,2491000,2960500,24998200,641700
2024-12-27,2745400.0,631800,9280700,517900,1397000,2650100.0,1195700,457000,724900,18891400,2947200,1842600,429000.0,728900,1094400,390400,1104800,2829500,32924000,1305500


In [46]:
# df1 = yf.download(['AAPL', 'AMD'], start='2020-01-01', end='2025-01-01')
# df1